This script creates the dataset for the empirical work in Alleo et al (2020) in R. All data is downloaded automatically from the internet except for police officers rate which comes from PORData. Advanced knowledge in browser automation is required to go around this issue and is beyond our scope. So pordata.xlsx must be copied to the working directory. After running the scripts the dataframe is exported to ISdataset.xlsx. 

In [164]:
import requests
from bs4 import BeautifulSoup
import eurostat
import pandas as pd
import numpy as np

In [91]:
def get_table (site, data_type):
    url=site
    s=requests.get(url).content
    if data_type == 1:
        data = pd.read_excel(s,skiprows=3).iloc[:30,]
        i=0
        for country in data['Country']:
            data['Country'][i]=data['Country'][i].strip('* ')
            i+=1
    elif data_type == 2:
        data = pd.read_excel(s,skiprows=3)
    elif data_type == 3:
        data = pd.read_csv(io.StringIO(s.decode('utf-8')))
    elif data_type == 4:
        data = pd.read_excel(s,skiprows=2)
    return data

In [11]:
URL = 'https://laendercode.net/en/3-letter-list.html'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

table = soup.find('table')
rows = table.find_all('td')
countries_abv3 = {}
for row in rows:
    if 'code/' in str(row):
        country_code = str(row).split('\">')
        country_code = str(country_code[0][-3:])
    elif ('country') in str(row):
        country_name = str(row).split('<')
        country_name = str(country_name[4][6:])
        countries_abv3.update({country_name:country_code.upper()})

In [13]:
URL = 'https://laendercode.net/en/2-letter-list.html'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')
data = []

table = soup.find('table')
rows = table.find_all('td')
countries_abv2 = {}
for row in rows:
    if 'code/' in str(row):
        country_code = str(row).split('\">')
        country_code = str(country_code[0][-3:])
    elif ('country') in str(row):
        country_name = str(row).split('<')
        country_name = str(country_name[4][6:])
        countries_abv2.update({country_name:country_code.strip('/').upper()})

In [18]:
data_gov_exp = eurostat.get_data('gov_10a_exp')

In [106]:
resp = requests.get('https://dw.euro.who.int/api/v3/export/HFA_426?format=xlsx')
url_alcohol = resp.json()['download_url']
url_list = pd.DataFrame([['od','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=DRD-31',1],
                        ['herbal_cann_kg','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=SZR-2-2',1],
                        ['resin_cann_kg','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=SZR-1-2',1],
                        ['heroin_kg','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=SZR-7-2',1],
                        ['cocaine_kg','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=SZR-10-2',1],
                        ['amph_kg','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=SZR-12-2',1],
                        ['ecs_un','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=SZR-15-2',1],
                        ['lsd_un','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=SZR-19-2',1],
                        ['cann_off','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=DLO-03-1',1],
                        ['heroin_off','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=DLO-03-4',1],
                        ['cocaine_off','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=DLO-03-7',1],
                        ['amph_off','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=DLO-03-13',1],
                        ['ecs_off','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=DLO-03-19',1],
                        ['lsd_off','https://www.emcdda.europa.eu/modules/sbdata/SBDataService2019.cfc?method=fetchxlsx&tableid=DLO-03-22',1],
                        ['alcohol','https://webgate.ec.europa.eu/dyna/echi/datatool/tab_4xls.cfm?indiclist=DHIND025010,&countries_box=Min,Avg,Max,BE,BG,CZ,DK,DE,EE,IE,EL,ES,FR,HR,IT,CY,LV,LT,LU,HU,MT,NL,AT,PL,PT,RO,SI,SK,FI,SE,UK,IS,NO,CH,ME,RS,MK,AL,TR,BA,&years_box=',4],
                        ['population','http://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=excel',2],
                        ['unemployment','http://api.worldbank.org/v2/en/indicator/SL.UEM.TOTL.ZS?downloadformat=excel',2],
                        ['education','http://api.worldbank.org/v2/en/indicator/SE.PRM.NENR?downloadformat=excel',2],
                        ['gdp','http://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.CD?downloadformat=excel',2],
                        ['gini','http://api.worldbank.org/v2/en/indicator/SI.POV.GINI?downloadformat=excel',2]])

In [110]:
i=0
while (i < len(url_list)):
    print(url_list[0][i])
    exec('{} = get_table(\'{}\', {})'.format(url_list[0][i], url_list[1][i], url_list[2][i]))
    i+=1

od


D:\users\F05434D\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


herbal_cann_kg
resin_cann_kg
heroin_kg
cocaine_kg
amph_kg
ecs_un
lsd_un
cann_off
heroin_off
cocaine_off
amph_off
ecs_off
lsd_off
alcohol
population
unemployment
education
gdp
gini


In [212]:
def merge ():
    countries = od['Country']
    years = [str(year) for year in range(2000,2018)]
    Columns = ['Country','Year','od','herbal_cann_kg','resin_cann_kg','heroin_kg','cocaine_kg','amph_kg',
               'ecs_un','cann_off','heroin_off','cocaine_off','amph_off','ecs_off','population','alcohol',
               'unemployment','gdp','gini','education','enforcement_exp','police_of_rate']
    por_data = pd.read_excel('pordata.xlsx', skiprows = 7)
    dataset = pd.DataFrame(columns=Columns)
    for country in countries:
        i=18
        j=0
        print('-',end='')
        for year in years:
            
            #Illegal substance observables
            OD=od.loc[od['Country'] == country].iloc[0,i]
            HERBAL_CANN_KG=herbal_cann_kg.loc[herbal_cann_kg['Country'] == country].iloc[0,i]
            RESIN_CANN_KG=resin_cann_kg.loc[resin_cann_kg['Country'] == country].iloc[0,i]
            HEROIN_KG=heroin_kg.loc[heroin_kg['Country'] == country].iloc[0,i]
            COCAINE_KG=cocaine_kg.loc[cocaine_kg['Country'] == country].iloc[0,i]
            AMPH_KG=amph_kg.loc[amph_kg['Country'] == country].iloc[0,i]
            ECS_UN=ecs_un.loc[ecs_un['Country'] == country].iloc[0,i]
            if i<16:
                CANN_OFF=cann_off.loc[cann_off['Country'] == country].iloc[0,i]
                HEROIN_OFF=heroin_off.loc[heroin_off['Country'] == country].iloc[0,i]
                COCAINE_OFF=cocaine_off.loc[cocaine_off['Country'] == country].iloc[0,i]
                AMPH_OFF=amph_off.loc[amph_off['Country'] == country].iloc[0,i]
                ECS_OFF=ecs_off.loc[ecs_off['Country'] == country].iloc[0,i]
            else:
                CANN_OFF=HEROIN_OFF=COCAINE_OFF=AMPH_OFF=ECS_OFF='NaN'
            
            #Control Variables
            if int(year) < 2017:
                inds = alcohol.apply(lambda x: x["Country"] == country and x["Year"] == int(year), axis=1)
                try: 
                    AC = float(alcohol[inds]["Value"])
                except:
                    AC = 'NaN'
            else:
                AC = 'NaN'
                
            if country == 'Czechia': 
                country2 = 'Czech Republic'
            elif country == 'Slovakia':
                country2 = 'Slovak Republic'
            else:
                country2 = country
            
            POP=population.loc[population['Country Name'] == country2].iloc[0,43+i]
            UNEMP=unemployment.loc[unemployment['Country Name'] == country2].iloc[0,43+i]
            GDP=gdp.loc[gdp['Country Name'] == country2].iloc[0,43+i]
            GINI=gini.loc[gini['Country Name'] == country2].iloc[0,43+i]
            EDU=education.loc[education['Country Name'] == country2].iloc[0,43+i]
            
            #Instruments
            if country == 'Slovakia':
                country2 = 'Slovakia'
            for item in data_gov_exp:
                if item[0] == 'PC_GDP' and item[4] == countries_abv2[country2] and item[2] == 'GF03' and item[1] == 'S13' and item[3] == 'TE':
                    EE = item[j+5]
                    
            if country2 == 'United Kingdom':
                POR = por_data['UK' + ' - ' + country2][j + 4]
            elif country2 != 'Turkey':
                POR = por_data[countries_abv2[country2] + ' - ' + country2][j + 4]
            else:
                POR = 'NaN'
            
            if POR == 0:
                POR = 'NaN'
                
            dataset.loc[len(dataset)] = [country,year,OD,HERBAL_CANN_KG,RESIN_CANN_KG,HEROIN_KG,COCAINE_KG,
                                         AMPH_KG,ECS_UN,CANN_OFF,HEROIN_OFF,COCAINE_OFF,AMPH_OFF,ECS_OFF,
                                         POP,AC,UNEMP,GDP,GINI,EDU,EE,POR]
            i-=1
            j+=1
            
    return dataset
        

In [213]:
dataset = merge()

------------------------------

In [214]:
dataset

,Country,Year,od,herbal_cann_kg,resin_cann_kg,heroin_kg,cocaine_kg,amph_kg,ecs_un,cann_off,...,amph_off,ecs_off,population,alcohol,unemployment,gdp,gini,education,enforcement_exp,police_of_rate
0,Austria,2000,132.0,1562.000,244.000,230.000,20.000,NaN,162093.0,NaN,...,NaN,NaN,8797566.0,13.2,5.499,47549.079040,29.7,88.61718,1.3,361.7
1,Austria,2001,117.0,282.000,138.000,288.000,108.000,NaN,256299.0,NaN,...,NaN,NaN,8736668.0,12.4,6.014,45237.805092,30.8,88.84197,1.4,353.6
2,Austria,2002,114.0,450.000,135.000,60.000,37.000,NaN,383451.0,NaN,...,NaN,NaN,8642699.0,12.5,5.723,44178.047378,30.5,88.59448,1.3,344
3,Austria,2003,133.0,553.000,239.000,43.000,58.000,NaN,422103.0,NaN,...,NaN,NaN,8546356.0,12.2,5.620,51717.495941,30.5,88.70387,1.3,327.9
4,Austria,2004,147.0,506.000,427.000,235.000,76.000,NaN,122663.0,20252,...,NaN,NaN,8479823.0,12.1,5.335,50716.708706,30.8,88.38265,1.3,331.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,United Kingdom,2013,1869.0,18705.000,1133.600,784.900,3561.500,1730.100,423000.0,72002,...,6342,3323,59987905.0,9.65,4.594,40290.313971,36.0,98.13312,1.3,NaN
536,United Kingdom,2014,1993.0,15549.720,7682.840,1220.500,3542.960,742.120,1149105.0,64527,...,6191,3177,59647577.0,9.69,4.807,34419.147915,NaN,99.90834,1.3,NaN
537,United Kingdom,2015,2313.0,31131.120,7807.300,882.150,4316.590,561.110,825516.0,56352,...,5365,3241,59370479.0,9.82,5.037,30056.586218,NaN,99.82263,1.3,NaN
538,United Kingdom,2016,2318.0,12615.186,6281.469,843.934,5697.481,1356.232,513259.0,51402,...,4299,2900,59119673.0,9.81,4.695,27744.506459,NaN,99.93639,1.3,NaN


In [215]:
dataset.describe(include = 'all')

,Country,Year,od,herbal_cann_kg,resin_cann_kg,heroin_kg,cocaine_kg,amph_kg,ecs_un,cann_off,...,amph_off,ecs_off,population,alcohol,unemployment,gdp,gini,education,enforcement_exp,police_of_rate
count,540,540,481.000000,498.000000,500.000000,514.000000,507.000000,3.960000e+02,4.960000e+02,371,...,323,324,5.400000e+02,540,540.000000,540.000000,436.000000,496.000000,540.000000,540
unique,30,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,278,...,180,178,NaN,361,NaN,NaN,NaN,NaN,NaN,449
top,Ireland,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,18,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90,...,90,90,NaN,37,NaN,NaN,NaN,NaN,NaN,53
mean,NaN,NaN,207.066528,4943.419282,23922.167959,577.551031,2456.128672,1.279178e+05,3.843917e+05,NaN,...,NaN,NaN,1.923513e+07,NaN,8.742711,29506.298107,31.652294,95.109879,1.723148,NaN
std,NaN,NaN,363.341248,14809.908929,96451.298969,1999.132803,6794.347567,1.243101e+06,1.035622e+06,NaN,...,NaN,NaN,2.405360e+07,NaN,4.355945,22058.991540,4.036440,4.349349,0.457104,NaN
min,NaN,NaN,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000e+00,1.200000e+01,NaN,...,NaN,NaN,3.900870e+05,NaN,1.805000,1621.242961,23.700000,82.184060,0.800000,NaN
25%,NaN,NaN,23.000000,91.000000,9.750000,6.000000,6.564000,3.000000e+00,6.015125e+03,NaN,...,NaN,NaN,4.229693e+06,NaN,5.755750,13137.535938,28.175000,93.310400,1.300000,NaN
50%,NaN,NaN,88.000000,375.000000,214.313000,59.000000,57.000000,5.595000e+01,3.525200e+04,NaN,...,NaN,NaN,8.834838e+06,NaN,7.728000,23621.906038,31.600000,96.277580,1.700000,NaN
75%,NaN,NaN,191.000000,3494.000000,3821.162500,329.990000,1313.000000,3.573970e+02,2.149950e+05,NaN,...,NaN,NaN,2.041008e+07,NaN,10.565500,42495.312200,34.700000,98.435435,2.000000,NaN


In [216]:
dataset.to_excel('ISdataset.xlsx', na_rep='NaN')